# Projeto Final

## Primeira entrega parcial

### Objetivo
Para essa entrega focaremos somente nas etapas de: 

- Escolher uma API da lista disponível; 
- Fazer a extração de pelo menos 3 tabelas;
- Criar um alerta de erro usando a função alerta() do desafio da aula Fundamentos de programação III~.

*ATENÇÃO:* _escolher apenas uma das APIs disponíveis._

### Formato
O entregável é um link do GitHub para um arquivo .py ou .ipynb com a resolução proposta.

Utilize o mesmo repositório criado na aula de GIT.   

### Observações

Não é necessário fazer tratamentos das tabelas, salvar no banco de dados ou documentação nesta etapa do projeto.

### Escolher uma API

APIs disponíveis:

- Brasil API - [Link](https://brasilapi.com.br/docs) (exemplo de url: https://brasilapi.com.br/api/banks/v1)

- Rest Coutries - [Link](https://restcountries.com/)  (exemplo de url: https://restcountries.com/v3.1/all)

- PokéAPI - [Link](https://pokeapi.co/) (exemplo de url: https://pokeapi.co/api/v2/pokemon)

# Código da Entrega

In [160]:
import requests
import pandas as pd
import sqlite3
import json
import urllib.parse
from plyer import notification

In [161]:
def salva_bd(df,nome_tabela,conn):     
    df.to_sql(nome_tabela,conn,if_exists='replace',index=False)

    return True

def carrega_bd(nome_tabela,conn):
    query = f'SELECT * FROM {nome_tabela}'
    df = pd.read_sql(query,conn)

    return df

In [162]:
holidays_raw_data = requests.get(f"https://brasilapi.com.br/api/feriados/v1/2024")

holidays_raw_data.content
holidays_raw_data

<Response [200]>

In [163]:
estados_raw_data = requests.get(f"https://brasilapi.com.br/api/ibge/uf/v1")

estados_raw_data.content
type(estados_raw_data.status_code)

int

In [164]:
cidade_raw_data = requests.get("https://brasilapi.com.br/api/cptec/v1/cidade/S%C3%A3o%20Paulo")

cidade_raw_data.content

b'[{"nome":"S\xc3\xa3o Paulo","id":244,"estado":"SP"},{"nome":"S\xc3\xa3o Paulo das Miss\xc3\xb5es","id":5019,"estado":"RS"},{"nome":"S\xc3\xa3o Paulo de Oliven\xc3\xa7a","id":5020,"estado":"AM"},{"nome":"S\xc3\xa3o Paulo do Potengi","id":5021,"estado":"RN"}]'

In [165]:
def valida_opcao_menu_inicial(opcao_menu_inicial):
        match opcao_menu_inicial:
            case '1':
                return listar_feriados()
            case '2':
                return listar_estados()
            case '3':
                return buscar_previsao()
            case _:
                print("Opção inválida.")
                return


def listar_feriados():
    ano = input("Favor informar o ano desejado")
    holidays_raw_data = requests.get(f"https://brasilapi.com.br/api/feriados/v1/{ano}")

    if holidays_raw_data.status_code != 200:
        return alerta(3, "https://brasilapi.com.br/api/feriados/v1/{ano}", "Extração")
    else:
        df_holidays = beautify_data(holidays_raw_data.content)

        #salva_bd(df_holidays,'feriados_nacionais',conn_holidays)
        display(df_holidays)
        
        return df_holidays


def listar_estados():
    estados_raw_data = requests.get("https://brasilapi.com.br/api/ibge/uf/v1")

    if estados_raw_data.status_code != 200:
        return alerta(3, "https://brasilapi.com.br/api/ibge/uf/v1", "Extração")
    
    else:
        df_estados = beautify_data(estados_raw_data.content)

        display(df_estados)
        
        #salva_bd(df_estados,'estados',conn_estados)
        
        return df_estados


def buscar_previsao():
    cidade = input("Informe a cidade para qual deseja saber a previsão do tempo.")
    urlencoded_cidade = urllib.parse.quote(cidade)
    
    days = input("Para quantos dias gostaria de ter a previsão?")

    cidade_previsao_raw_data = requests.get(f"https://brasilapi.com.br/api/cptec/v1/cidade/{urlencoded_cidade}")

    if cidade_previsao_raw_data.status_code != 200:
        return alerta(3, "https://brasilapi.com.br/api/cptec/v1/cidade/{urlencoded_cidade}", "Extração")
    
    else:
        df_cidade_previsao = beautify_data(cidade_previsao_raw_data.content)
        #salva_bd(df_cidades,'previsao_por_cidade',conn_previsao)
        display(df_cidade_previsao)
        display(df_cidade_previsao.iloc[0]['id'])
        
        cityCode = int(df_cidade_previsao.iloc[0]['id'])
    
        previsao_raw_data = requests.get(f"https://brasilapi.com.br/api/cptec/v1/clima/previsao/{cityCode}/{days}")
        display(previsao_raw_data
                )
        df_previsao = beautify_data(previsao_raw_data.content)
        display(df_previsao)    
            
        return previsao_raw_data


def beautify_data(raw_data):
    decoded_data = raw_data.decode('utf-8')
    working_data = json.loads(decoded_data)
    dataframe = pd.DataFrame(working_data)
    return dataframe


def alerta(nivel, base, etapa):
    alerta_nivel = ""
    match nivel:
        case 1:
            alerta_nivel = "Baixo"
        case 2:
            alerta_nivel = "Médio"
        case 3:
            alerta_nivel = "Alto"
            
    notification.notify(
        title=f"ERRO: Alerta {alerta_nivel} ", 
        message=f"Falha no carregamento da base {base} na etapa {etapa}", 
        app_name="", 
        timeout=10
)


In [166]:
conn_holidays = sqlite3.connect('holidays.db')
conn_estados = sqlite3.connect('estados.db')
conn_previsao = sqlite3.connect('previsao.db')

print("Boas-vindas ao SIPD - Sistema integrado de planejamento divertido")

opcao_menu_inicial = input(
    "Boas-vindas ao SIPD - Sistema integrado de planejamento divertido\n Escolha uma opção:\n 1 - Feriados Nacionais\n 2 - Roleta de Estados para Visitar\n 3 - Previsão do Tempo")

valida_opcao_menu_inicial(opcao_menu_inicial)

conn_holidays.close()
conn_estados.close()
conn_previsao.close()

Boas-vindas ao SIPD - Sistema integrado de planejamento divertido


,nome,id,estado
0,São Paulo,244,SP
1,São Paulo das Missões,5019,RS
2,São Paulo de Olivença,5020,AM
3,São Paulo do Potengi,5021,RN


244

<Response [200]>

,cidade,estado,atualizado_em,clima
0,São Paulo,SP,2024-05-27,"{'data': '2024-05-27', 'condicao': 'c', 'condi..."
1,São Paulo,SP,2024-05-27,"{'data': '2024-05-28', 'condicao': 'pn', 'cond..."
2,São Paulo,SP,2024-05-27,"{'data': '2024-05-29', 'condicao': 'pn', 'cond..."
3,São Paulo,SP,2024-05-27,"{'data': '2024-05-30', 'condicao': 'pn', 'cond..."
4,São Paulo,SP,2024-05-27,"{'data': '2024-05-31', 'condicao': 'pn', 'cond..."
5,São Paulo,SP,2024-05-27,"{'data': '2024-06-01', 'condicao': 'pn', 'cond..."
